In [ ]:
#bow + tfidf　による特徴語抽出を行い上位20単語を抜き出し
#wmdやcos類似度によって類似動画の推薦を行う
#類似動画かどうか自動判定を用いて判定を行い
#それぞれの推薦方法による評価を行う

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#!pip install japanize-matplotlib
import japanize_matplotlib
import glob

In [2]:
rank_df = pd.read_csv("../csv_file/rank3_df.csv",index_col=0)
meta_df = pd.read_csv("../csv_file/meta3_df.csv",index_col=0)

In [4]:
meta_df["category"].value_counts()

game           204
music_sound    149
anime          147
Name: category, dtype: int64

In [10]:
#rank_dfの詳細取得
def get_detail(rank_df):
    #長さ
    print(f"長さ : {len(rank_df)}")
    #各印象ごとの数
    #print("各印象ごとの数")
    #for index, value in rank_df['imp'].value_counts().iteritems():
        #print(index, ': ', value)
    #推薦された動画が全て似ていた動画の数と割合
    all_1 = len(rank_df[(rank_df["1_mark"]== 1)&(rank_df["2_mark"]== 1)&(rank_df["3_mark"]== 1)])
    print(f"推薦動画が全て似ている動画数 : {all_1}")
    print(f"推薦動画が全て似ている動画の割合 : {all_1 / len(rank_df)}")
    #一つ目に推薦された動画の似ている割合
    first = len(rank_df[(rank_df["1_mark"]== 1)])
    print(f"文書間距離が1番目近い動画の似ている動画数 : {first}")
    print(f"文書間距離が1番目近い動画の似ている動画の割合 : {first / len(rank_df)}")
    #二つ目に推薦された動画の似ている割合  
    second = len(rank_df[(rank_df["2_mark"]== 1)])
    print(f"文書間距離が2番目に近い動画の似ている動画数 : {second}")
    print(f"文書間距離が2番目に動画の似ている動画の割合 : {second / len(rank_df)}")
    #三つ目に推薦された動画の似ている割合    
    third = len(rank_df[(rank_df["3_mark"]== 1)])
    print(f"文書間距離が3番目に近い動画の似ている動画数 : {third}")
    print(f"文書間距離が3番目に動画の似ている動画の割合 : {third / len(rank_df)}")
    #推薦された動画が全く似ていない動画の数と割合
    all_0 = len(rank_df[(rank_df["1_mark"]== 0)&(rank_df["2_mark"]== 0)&(rank_df["3_mark"]== 0)])
    print(f"推薦された動画が全く似ていない動画の数 : {all_0}")
    print(f"推薦された動画が全く似ていない動画の割合 : {all_0 / len(rank_df)}")

In [15]:
#meta_dfのタグ調整
meta_df = pd.read_csv("meta3_df.csv",index_col=0)
meta_df = meta_df.rename(columns={'tags': 'tags_org'})
meta_df.insert(8, 'tags_check', 0)
for m in range(len(meta_df)): #誤差2.51% #元タグ誤差6.88%
    tag = meta_df["tags_org"].iloc[m].split(" ")
    text = meta_df["tags_org"].iloc[m]
    title = meta_df["title"].iloc[m]
    if "音楽" in tag:
        tag.remove("音楽")
    if "アニメ" in tag:
        tag.remove("アニメ") 
    if "ゲーム" in tag:
        tag.remove("ゲーム") 
    if "実況プレイ動画" in tag:
        tag.remove("実況プレイ動画")
    if (("MAD" in text) | ("MAD" in title) | ("RED_ZONE" in tag)|("ブロじろう" in tag)):
        tag.append("MAD")
    if (("桜ノ雨" in tag) | ("旅立チノ時" in tag) | ("卒業" in text)):
        tag.append("卒業")
    if ("BGM" in title) | ("メドレー" in title) | ("BGM" in text) | ("メドレー" in text):
        tag.append("BGM")
    if ("マリオ" in title) | ("マリオ" in text):
        tag.append("mario")
    if ("ポケモン" in title) | ("ポケモン" in text):
        tag.append("pokemon")
    if ("声優" in title) | ("声優" in text):
        tag.append("voice_actor")
    if ("コナン" in text) | ("高山ミナミ" in text):
        tag.append("conan")
    if ("ノンノンビヨリ" in tag) | ("ケモノフレンズ" in tag) | ("テーキュウ" in tag):
        tag.append("癒しanime")
    if ("オリジナル" in text):
        tag.append("original")
    if ("マドカ☆マギカ" in text):
        tag.append("マドカ☆マギカ")
    if ("Fate" in text)|("FATE" in text):
        tag.append("fate")
    if ("アニソン" in text)|("OP" in text):
        tag.append("anime_song")
    if ("厨ポケ狩り" in title)|("モコウ" in text):
        tag.append("mokou")
    if ("腹筋崩壊" in text)|("吹イタラ" in text)|("元気ガ出ル動画" in tag):
        tag.append("funny")
    if ("ロボット" in text):
        tag.append("robot")
    if ("ドレドドミ:ガチホモポケモン" in tag):
        tag.remove("pokemon")
    if ("ユックリ実況プレイ" in tag):
        tag.remove("ユックリ実況プレイ")
    if ("コノスバ" in text)|("リゼロ" in text):
        tag.append("Different_world")
    if ("スマブラ" in text)|("スマブラ" in tag):
        tag.append("smash_brothers")
    
    meta_df["tags_check"].iloc[m] = " ".join(tag)

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [16]:
#タグデータで評価する方法
def group_check_tag(meta_num1, meta_num2):
    group1 = meta_df["tags_check"].iloc[meta_num1].split(" ")
    group2 = meta_df["tags_check"].iloc[meta_num2].split(" ")
    check = list(set(group1) & set(group2))
    if check:
        return True
    else:
        return False

In [4]:
movie_df = pd.read_csv("movie3_df_list.csv",index_col=0)
for i in range(len(movie_df)):
    movie_df["content"].iloc[i] = movie_df["content"].iloc[i].split(" ")
movie_df

,id,content
0,moviedata2/sm17961470.jsonl,"[きくお, 幸福, 鬱くしい, 好き, 地獄, 天国, 綺麗, 歌詞, ミク, ダブステ, ..."
1,moviedata2/sm12419220.jsonl,"[シンジ, もこう, ポケモン, 厳選, 先生, 廃人, ゾロア, ポカブ, つり, マジレ..."
2,moviedata2/sm5430176.jsonl,"[アリス, レン, 不思議, 番目, おk, リン, ミク, カイト, 真っ赤, 一時停止,..."
3,moviedata2/so15281450.jsonl,"[アイマス, ナンバーワン, うたプリ, 美希, ここ, 周年, 公式, op, akb, ..."
4,moviedata2/sm2069100.jsonl,"[勉強, テスト, 明日, 集中, 休憩, 数学, レポート, 時間, 今日, みんな, 英..."
...,...,...
495,moviedata2/sm5408420.jsonl,"[ルル, ルルーシュ, 太平洋, ひで, ギアス, ローゼン, 触手, スザク, みのりん,..."
496,moviedata2/sm10116060.jsonl,"[美琴, 黒子, 俺の嫁, 職人, 木山, 初春, レールガン, 禁書, 上条さん, そげぶ..."
497,moviedata2/sm20715461.jsonl,"[サシャ, 巨人, エレン, ベルトルト, ミーナ, フラグ, 駆逐, ネタバレ, 駆逐系男..."
498,moviedata2/sm3120540.jsonl,"[一番, シャーリー, ユフィ, ルル, 解読不能, モザイクカケラ, ルルーシュ, ギアス..."


In [6]:
import gensim
from gensim.models import word2vec 
import logging
import sys
import gensim.downloader as gendl
import pyemd

In [7]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
model = KeyedVectors.load_word2vec_format(
    datapath("/mnt/data1/home/ooga/models/jawiki.all_vectors.300d.txt"),
    binary=False
)

In [8]:
#WMDによる推薦
rank_df=pd.DataFrame(columns=["1_num", "1_score", "1_mark", "2_num", "2_score", "2_mark", "3_num", "3_score", "3_mark"])
target = 0
rank = []
for x in range(len(movie_df)):
    target = x
    rank = []
    for i in range(len(movie_df)):
        if target != i:
            sim_value = model.wmdistance(movie_df.iloc[target,1], movie_df.iloc[i,1])
            rank.append((sim_value, i))
    ld = sorted(rank)
    rank_df.loc[x]=[ld[0][1], ld[0][0], 0, ld[1][1], ld[1][0], 0, ld[2][1], ld[2][0], 0]
    if(x % 10 == 0):
        print(x)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490


In [18]:
#その他推薦
#cos類似度による推薦
rank_df=pd.DataFrame(columns=["1_num", "1_score", "1_mark", "2_num", "2_score", "2_mark", "3_num", "3_score", "3_mark"])
target = 0
rank = []
#ベクトル作成
def avg_feature_vector(li, model, num_features=300):
    feature_vec = np.zeros((num_features,), dtype="float32") # 特徴ベクトルの入れ物を初期化
    feature_vec_ins = np.zeros(300, dtype="float32") # 特徴ベクトルの入れ物を初期化
    for word in li:
        if word in model:#辞書にない場合は省く
            feature_vec = np.add(feature_vec, model[word])
    if len(li) > 0:
        feature_vec = np.divide(feature_vec, len(li))
    return feature_vec

#cos類似度計算
from scipy import spatial
def sentence_similarity(list_1, list_2):
    return 1 - spatial.distance.cosine(list_1, list_2)

#cos類似度で判定
for x in range(len(movie_df)):
    target = x
    rank = []
    for i in range(len(movie_df)):
        if target != i:
            #計算
            sim_value = sentence_similarity(avg_feature_vector(movie_df["content"].iloc[target][0:10], model),avg_feature_vector(movie_df["content"].iloc[i][0:10], model))
            rank.append((sim_value, i))
    ld = sorted(rank,reverse=True)
    rank_df.loc[x]=[ld[0][1], ld[0][0], 0, ld[1][1], ld[1][0], 0, ld[2][1], ld[2][0], 0]
    if(x % 10 == 0):
        print(x)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490


In [ ]:
#ユークリッド距離
def euclidean_distance(x, y):   
    return np.sqrt(np.sum((x - y) ** 2))

#np.linalg.norm関数で計算可能

rank_df=pd.DataFrame(columns=["1_num", "1_score", "1_mark", "2_num", "2_score", "2_mark", "3_num", "3_score", "3_mark"])
target = 0

#ユークリッド距離で計算
for x in range(len(movie_df)):
    target = x
    rank = []
    for i in range(len(movie_df)):
        if target != i:
            #計算
            sim_value = euclidean_distance(avg_feature_vector(movie_df["content"].iloc[target],model), avg_feature_vector(movie_df["content"].iloc[i], model))
            rank.append((sim_value, i))
    ld = sorted(rank)
    rank_df.loc[x]=[ld[0][1], ld[0][0], 0, ld[1][1], ld[1][0], 0, ld[2][1], ld[2][0], 0]
    if(x % 10 == 0):
        print(x)

In [19]:
#タグで評価
for mark in range(len(rank_df)):
    if(group_check_tag(mark, int(rank_df["1_num"].iloc[mark]))):
        rank_df["1_mark"].iloc[mark] = 1
    if(group_check_tag(mark, int(rank_df["2_num"].iloc[mark]))):
        rank_df["2_mark"].iloc[mark] = 1
    if(group_check_tag(mark, int(rank_df["3_num"].iloc[mark]))):
        rank_df["3_mark"].iloc[mark] = 1

In [20]:
get_detail(rank_df) #cos類似度による推薦

長さ : 500
推薦動画が全て似ている動画数 : 253
推薦動画が全て似ている動画の割合 : 0.506
文書間距離が1番目近い動画の似ている動画数 : 402
文書間距離が1番目近い動画の似ている動画の割合 : 0.804
文書間距離が2番目に近い動画の似ている動画数 : 335
文書間距離が2番目に動画の似ている動画の割合 : 0.67
文書間距離が3番目に近い動画の似ている動画数 : 306
文書間距離が3番目に動画の似ている動画の割合 : 0.612
推薦された動画が全く似ていない動画の数 : 66
推薦された動画が全く似ていない動画の割合 : 0.132
